In [1]:
from sdv.tabular import CTGAN


/home/diogoaos/workspace/iseg/iseg_07_ppai_group/exer2/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sdv
import pandas as pd

In [7]:
data = pd.read_csv('data/loan_defaults.csv')

In [9]:
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,has_missed_loans
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,False
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,False
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,False
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,False
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,True


In [15]:
from sdv.constraints import Unique, Inequality, create_custom_constraint

In [14]:
# num dias a trabalhar > num dias vivo
age_gt_age_when_joined_constraint = Inequality(
    low_column_name='DAYS_BIRTH',
    high_column_name='DAYS_EMPLOYED'
    )

In [177]:
def no_children_working_is_valid(column_names, data, working_age_years):
    # assumes column_names[0] has column of DAYS_BIRTH
    # assumes column_names[1] has column of DAYS_EMPLOYED
    days_birth = data[column_names[0]]
    days_employed = data[column_names[1]]
    
    days_employed_working_age = -working_age_years * 365
    
    return (days_birth - days_employed) <  days_employed_working_age


NoChildWorkerConstraint = create_custom_constraint(
    is_valid_fn=no_children_working_is_valid
)

In [179]:
constraints = [
    age_gt_age_when_joined_constraint,
    NoChildWorkerConstraint(column_names=['DAYS_BIRTH', 'DAYS_EMPLOYED'], working_age_years=16)
]

In [181]:
model = CTGAN(primary_key='ID',
             constraints=constraints)

In [ ]:
model.fit(data)

/home/diogoaos/workspace/iseg/iseg_07_ppai_group/exer2/venv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/diogoaos/workspace/iseg/iseg_07_ppai_group/exer2/venv/lib/python3.8/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
/home/diogoaos/workspace/iseg/iseg_07_ppai_group/exer2/venv/lib/python3.8/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
/home/diogoaos/workspace/iseg/iseg_07_ppai_group/exer2/venv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller